In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [61]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [62]:
DATA_DIR = "/content/drive/My Drive/NLP/toxic-comment-classification/data"
MODEL_DIR = "/content/drive/My Drive/NLP/toxic-comment-classification/models"
RESULTS_DIR = "/content/drive/My Drive/NLP/toxic-comment-classification/results"

In [63]:
test = pd.read_csv(os.path.join(DATA_DIR,"test.csv"))

In [64]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [65]:


print("Check for missing values in Test dataset")
null_check=test.isnull().sum()
print(null_check)
print("filling NA with \"unknown\"")

test["comment_text"].fillna("unknown", inplace=True)


Check for missing values in Test dataset
id              0
comment_text    0
dtype: int64
filling NA with "unknown"


In [66]:
test_labels = pd.read_csv(os.path.join(DATA_DIR,"test_labels.csv"))

In [67]:
test_labels.shape

(153164, 7)

In [68]:


print("Check for missing values in Test dataset")
null_check=test.isnull().sum()
print(null_check)
print("filling NA with \"unknown\"")

test["comment_text"].fillna("unknown", inplace=True)


Check for missing values in Test dataset
id              0
comment_text    0
dtype: int64
filling NA with "unknown"


In [69]:
#after the competiton the test labels were released and the unused samples for testing were given values of -1. So I wrote code to eliminate those sentences and their corresponding 
#labels from the test set

test_set = test.join(test_labels.set_index("id"),on="id")

In [70]:
test_set.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1


In [71]:
test_set.shape

(153164, 8)

In [72]:
test_set = test_set[test_set.obscene!=-1]

In [73]:
print(test_set.shape)


(63978, 8)


In [74]:
test_set.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0


In [75]:
data = test_set
if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
data['comment_text'] = data['comment_text'].str.lower()
data['comment_text'] = data['comment_text'].apply(cleanHtml)
data['comment_text'] = data['comment_text'].apply(cleanPunc)
data['comment_text'] = data['comment_text'].apply(keepAlpha)

In [78]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)
data['comment_text'] = data['comment_text'].apply(removeStopWords)

In [79]:
test_set = data

In [80]:
test_set.shape

(63978, 8)

In [ ]:
test_set.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,yo bitch ja rule succes youll ever what hate s...,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,rfc titl fine imo,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,sourc zaw ashton lapland,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,look back sourc inform updat correct form gues...,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,dont anonym edit articl all,-1,-1,-1,-1,-1,-1


In [81]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = test_set[list_classes].values
list_sentences_test = test_set["comment_text"]

In [82]:
print(y.shape)
print(list_sentences_test.shape)

(63978, 6)
(63978,)


In [83]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_test))
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [84]:
maxlen = 200
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)
print(y.shape)
print(X_test.shape)

(63978, 6)
(63978, 200)


In [85]:
model = tf.keras.models.load_model(os.path.join(MODEL_DIR,"lstm-custom-embeddings-v4.hdf5"))

In [86]:
model.evaluate(X_test,y)

2000/2000 [==============================] - 137s 69ms/step - loss: 0.3194 - accuracy: 0.9976


[0.319404274225235, 0.9976085424423218]

In [87]:
y_pred = model.predict(X_test,batch_size=1024,verbose=1)

63/63 [==============================] - 84s 1s/step


In [ ]:
submission = pd.read_csv(os.path.join(RESULTS_DIR,"sample_submission.csv"))
print(submission.shape)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv(os.path.join(RESULTS_DIR,'submission.csv'), index=False)